In [149]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, Span
from bokeh.io import curdoc
from bokeh.themes import built_in_themes

from bokeh.layouts import row
import time
import pytz
import zoneinfo
from pytz import timezone

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [1]:
#change width of current notebook cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [73]:
#get existing database health_measure.description id and var_datetime_utc ***TODO Make this filter on user*****
base_query_health_description=db.session.query(Health_description)
df_health_description=pd.read_sql(str(base_query_health_description),db.session.bind) 
#correct column names
colNames=[i[len('health_description_'):] for i in list(df_health_description.columns)]
col_names_dict={i:j for i,j in zip(list(df_health_description.columns),colNames)}
df_health_description.rename(columns=col_names_dict, inplace=True)


In [74]:
#make dataframe of all values in chart ---Filter here
df1=df_health_description.loc[(df_health_description.metric1_carido<100) ]
#sort df1 by date
df1=df1.sort_values(by=['datetime_of_activity'])

#make lists for each data type
#1 time (obs_x)
obs_x1=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df1.datetime_of_activity]
#2 heart rate (obs_y1)
obs_y1=df1.metric1_carido
obs_y1=[ plot_text_format(i) for i in obs_y1]

In [75]:
df2=df_health_description.loc[(df_health_description.metric1_carido.isnull()) & (df_health_description.var_activity=='Activity') ]
#1 time (obs_x)
obs_x2=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df2.datetime_of_activity]
#3 activity (obs_y2)
obs_y2=[i if i=='Activity' else '' for i in df2.var_activity ]

In [83]:
# date1=datetime.datetime.strptime(df1.datetime_of_activity[2],'%Y-%m-%d %H:%M:%S.%f')
date2=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[-1],'%Y-%m-%d %H:%M:%S.%f')
date1=date2 - timedelta(days=7)
date2=date2 + timedelta(days=1)

In [85]:
date1

datetime.datetime(2021, 8, 13, 14, 4, 9, 641000)

In [115]:
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date1,date2),y_range=(-10,90),width=800, height=300)

#add cardio_metric1
source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1))
glyph1 = Text(text="text", text_color="#d6fbf7")
fig1.add_glyph(source1, glyph1)

#add activity data
source2 = ColumnDataSource(dict(x=obs_x2, y=[80], text=obs_y2))
glyph2 = Text(text="text", text_color="red")
#add line for activity data
line_start_time=time.mktime(obs_x2[0].timetuple())*1000
important_time = Span(location=line_start_time, dimension='height', line_color='red', line_dash='dashed', line_width=3)

line_start_time2=time.mktime((obs_x2[0]+timedelta(days=1)).timetuple())*1000
important_time2 = Span(location=line_start_time2, dimension='height', line_color='blue', line_dash='dashed', line_width=3)

fig1.add_layout(important_time)
fig1.add_layout(important_time2)

fig1.add_glyph(source2, glyph2)

curdoc().theme = 'contrast'


show(fig1)

In [110]:
line_date2=obs_x2[0]+timedelta(days=1)
line_date2

datetime.datetime(2021, 8, 17, 15, 0)

In [112]:
obs_x2[0]

datetime.datetime(2021, 8, 16, 15, 0)

In [58]:
def plot_text_format(x):
    return ('%.1f' % x).rstrip('0').rstrip('.')

In [156]:
time_var=datetime.datetime.now()
time_var.date().strftime("%Y-%m-%d")

'2021-08-22'

In [150]:
au_tz = timezone('US/Eastern')
time_var=datetime.datetime.now(tz=au_tz)
time_var.time().strftime("H:%M")


datetime.datetime(2021, 8, 22, 12, 43, 45, 729132, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>)

In [144]:
timezone_list=zoneinfo.available_timezones()
timezone_list

{'Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara',
 'Africa/Asmera',
 'Africa/Bamako',
 'Africa/Bangui',
 'Africa/Banjul',
 'Africa/Bissau',
 'Africa/Blantyre',
 'Africa/Brazzaville',
 'Africa/Bujumbura',
 'Africa/Cairo',
 'Africa/Casablanca',
 'Africa/Ceuta',
 'Africa/Conakry',
 'Africa/Dakar',
 'Africa/Dar_es_Salaam',
 'Africa/Djibouti',
 'Africa/Douala',
 'Africa/El_Aaiun',
 'Africa/Freetown',
 'Africa/Gaborone',
 'Africa/Harare',
 'Africa/Johannesburg',
 'Africa/Juba',
 'Africa/Kampala',
 'Africa/Khartoum',
 'Africa/Kigali',
 'Africa/Kinshasa',
 'Africa/Lagos',
 'Africa/Libreville',
 'Africa/Lome',
 'Africa/Luanda',
 'Africa/Lubumbashi',
 'Africa/Lusaka',
 'Africa/Malabo',
 'Africa/Maputo',
 'Africa/Maseru',
 'Africa/Mbabane',
 'Africa/Mogadishu',
 'Africa/Monrovia',
 'Africa/Nairobi',
 'Africa/Ndjamena',
 'Africa/Niamey',
 'Africa/Nouakchott',
 'Africa/Ouagadougou',
 'Africa/Porto-Novo',
 'Africa/Sao_Tome',
 'Africa/Timbuktu',
 'Africa/

In [158]:
activity_date='2021-08-22';activity_time='20:39'
date_time_obj = datetime.datetime.strptime(activity_date+activity_time, '%Y-%m-%d%H:%M')
date_time_obj

datetime.datetime(2021, 8, 22, 20, 39)

In [165]:
#get timezeon difference in minutes from utc
au_tz = timezone('Europe/Paris')
utc_time=datetime.datetime.now(tz=pytz.UTC)
cur_zone_time=datetime.datetime.now(tz=au_tz)
duration=(utc_time-cur_zone_time)
duration.total_seconds()

0.0

In [166]:
utc_time

datetime.datetime(2021, 8, 22, 18, 57, 12, 782422, tzinfo=<UTC>)

In [174]:
cur_zone_time.utcoffset().total_seconds()/60

120.0

In [128]:
base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)

In [141]:
base_query_health_description=db.session.query(Health_description).filter(Health_description.var_activity=="YOGA")
# print(str(base_query_health_description)[:-1]+str("YOGA"))
filter_var="""
"YOGA"
"""
df_health_description1=pd.read_sql(str(base_query_health_description)[:-1]+filter_var,db.session.bind)

In [143]:
df_health_description1.head()

,health_description_id,health_description_datetime_of_activity,health_description_var_activity,health_description_var_type,health_description_var_periodicity,health_description_var_timezone_utc_delta_in_mins,health_description_time_stamp_utc,health_description_user_id,health_description_source_filename,health_description_metric1_carido,health_description_metric2_session_duration,health_description_metric3,health_description_metric4,health_description_metric5
0,1,2021-06-09 12:08:34.635000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:51.668660,1,training-session-2021-06-09-6138678344-024ee81...,21.733782,359.0,None,None,None
1,13,2021-03-26 11:03:26.601000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:52.000689,1,training-session-2021-03-26-5835743489-0b1602e...,35.388216,666.0,None,None,None
2,17,2021-08-09 14:18:16.796000,YOGA,heart rate,seconds,120.0,2021-08-22 15:13:52.132196,1,training-session-2021-08-09-6362409969-0fb4326...,53.317569,879.0,None,None,None
3,18,2021-05-18 11:32:12.553000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:52.169951,1,training-session-2021-05-18-6052767144-102889e...,11.919939,728.0,None,None,None
4,19,2021-04-25 12:47:06.940000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:52.185581,1,training-session-2021-04-25-5967960349-103a12b...,20.440003,622.0,None,None,None
